Hace falta algo que indique con qué entorno vamos a trabajar

Importar lo que hace falta

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

In [2]:
data_url = "../Data/properatti.csv"

In [3]:
data = pd.read_csv(data_url, encoding="utf-8")

In [4]:
#limpiamos los que NaN en el precio
data = data.dropna(axis=0, how='any', subset=['price_aprox_usd'])

In [5]:
#funcion para borrar outliers.
def borrar_outliers(data, columnas):
    """Solo recibo columnas con valores numericos. 
    Las columns van en forma de tupla"""
    cols_limpiar = columnas
    mask=np.ones(shape=(data.shape[0]), dtype=bool)

    for i in cols_limpiar:
        
        #calculamos cuartiles, y valores de corte
        Q1=data[i].quantile(0.25)
        Q3=data[i].quantile(0.75)
        RSI=Q3-Q1
        max_value=Q3+1.5*RSI
        min_value=Q1-1.5*RSI
        
        #ajusto el min value a mano... no puede ser negativo.
        min_value=10
        
        #filtramos por max y min
        mask=np.logical_and(mask, np.logical_and(data[i]>=min_value, data[i]<=max_value))
    return data[mask]

In [6]:
def regex_to_bool(col, reg) :
    u"""Returns a series with boolean mask result of apply the regular expresion to the column
    col : column where to apply regular expresion
    reg : regular expresion compiled
    """
    
    serie = col.apply(lambda x : x if x is np.NaN else reg.search(x))
    serie = serie.apply(lambda x : x is not None)
   
    return serie

In [7]:
def regex_to_ones(col, reg, fill = 0) :
    u"""Returns a series with ones or other value result of apply the regular expresion to the column
    the value of one will be when the regular expression search() method found a match
    the fill value (default to 0) will be when the regular expression serach() method did not found a match
    col : column where to apply regular expresion
    reg : regular expresion compiled
    """
    
    serie = col.apply(lambda x : x if x is np.NaN else reg.search(x))
    serie = serie.apply(lambda x : 1 if x is not None else fill)
   
    return serie

In [8]:
_pattern = 'cochera|garage|auto'
_express = re.compile(_pattern, flags = re.IGNORECASE)

work = regex_to_ones(data['description'], _express)
data['cochera'] = work

In [9]:
_pattern = 'piscina|pileta'
_express = re.compile(_pattern, flags = re.IGNORECASE)

work = regex_to_ones(data['description'], _express)
data['pileta'] = work

In [10]:
_pattern = 'parrilla'
_express = re.compile(_pattern, flags = re.IGNORECASE)

work = regex_to_ones(data['description'], _express)
data['parrilla'] = work

In [11]:
_pattern = 'balcon'
_express = re.compile(_pattern, flags = re.IGNORECASE)

work = regex_to_ones(data['description'], _express)
data['balcon'] = work

In [18]:
# Crear una categoría númerica (en base 2) de acuerdo a los valores individuales (es dependiente de la posición)

data['amenities'] = data['cochera']*8 + data['pileta']*4 + data['parrilla']*2 + data['balcon']

In [19]:
data[['cochera', 'pileta', 'parrilla', 'balcon', 'amenities']]

,cochera,pileta,parrilla,balcon,amenities
0,0,0,0,0,0
1,1,0,0,0,8
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
121215,1,1,1,0,14
121216,1,1,1,0,14
121217,1,0,1,1,11
121218,0,0,0,0,0


In [14]:
data['amenities'].describe()

count    100810.000000
mean          5.692749
std           5.160266
min           0.000000
25%           0.000000
50%           6.000000
75%          10.000000
max          15.000000
Name: amenities, dtype: float64

In [15]:
data['amenities'].value_counts()

0     33437
8     18248
14     9069
10     7092
12     6101
2      4777
6      4366
1      4239
9      3412
4      3354
15     2010
13     1464
11     1349
3       869
5       549
7       474
Name: amenities, dtype: int64